# Exploring Spark with Pandas


Using pandas examples, convert the analysis to pyspark. This is useful if you discover your data grows too large for your tooling.

The purpose of this notebook is to familiarise yourself you the pyspark API. You are welcome to use the R version of this if you wish. As long as you are able to obtain the correct results. We will be using python in this notebook as it is quite widely used through data science and the community is very large.



#### Firstly, let's get our spark session

In [ ]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.2 LTS
Release:	22.04
Codename:	jammy


In [ ]:
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [498 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,014 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [998 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,260 kB]
Hit:13 https:

In [ ]:
# Install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# get spark
VERSION='3.5.0'
!wget https://dlcdn.apache.org/spark/spark-$VERSION/spark-$VERSION-bin-hadoop3.tgz

--2023-09-21 18:14:01--  https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400395283 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.0-bin-hadoop3.tgz’

spark-3.5.0-bin-had 100%[===================>] 381.85M  46.3MB/s    in 5.6s    

2023-09-21 18:14:07 (68.0 MB/s) - ‘spark-3.5.0-bin-hadoop3.tgz’ saved [400395283/400395283]



In [ ]:
# decompress spark
!tar xf spark-$VERSION-bin-hadoop3.tgz

# install python package to help with system paths
!pip install -q findspark

In [ ]:
# Let Colab know where the java and spark folders are

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-{VERSION}-bin-hadoop3"

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
import pandas as pd
spark = SparkSession.builder.appName("local[*]").getOrCreate()

# Questions

Answer the following questions by porting the pandas code to the Spark API



# Question 1


> On what day do most crashes occcur?

In [ ]:
# Pandas
nyc_small.groupby('CRASH DATE')['CRASH DATE'].count().sort_values(ascending=False).head(5)

In [ ]:
### Spark?

from pyspark.sql.functions import col

# Group by 'CRASH DATE', count, and order by count in descending order
result = sdf_small.groupBy(['CRASH DATE']).agg({'CRASH DATE': 'count'})\
                  .orderBy(col('count(CRASH DATE)').desc()) \
                  .limit(5)

# Show the result
result.show()


+----------+-----------------+
|CRASH DATE|count(CRASH DATE)|
+----------+-----------------+
|11/15/2018|              228|
|01/21/2014|              219|
|05/19/2017|              205|
|06/21/2018|              196|
|12/15/2017|              194|
+----------+-----------------+



# Question 2

> _Where do most crashes occur?_

In [ ]:
nyc_small.groupby('BOROUGH')['BOROUGH'].count().sort_values(ascending=False).head(5)

In [ ]:
## Spark?

from pyspark.sql.functions import col

result = sdf_small.groupBy('BOROUGH').agg({'BOROUGH': 'count'}) \
                  .orderBy(col('count(BOROUGH)').desc()) \
                  .limit(5)

result.show()


+---------+--------------+
|  BOROUGH|count(BOROUGH)|
+---------+--------------+
|         |        126088|
| BROOKLYN|         88569|
|   QUEENS|         74822|
|MANHATTAN|         62911|
|    BRONX|         41229|
+---------+--------------+



 # Question 3

 > What is the most common cause of accident in 'QUEENS'

In [ ]:
nyc_small[(nyc_small.BOROUGH == 'QUEENS')]['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()

# you can also use a group by (to avoid the pandas value_counts function)

nyc_small[(nyc_small.BOROUGH == 'QUEENS')].groupby(
    'CONTRIBUTING FACTOR VEHICLE 1'
)['CONTRIBUTING FACTOR VEHICLE 1'].count().sort_values(ascending=False).head(5)

In [ ]:
## Spark?

from pyspark.sql.functions import col, count

result = sdf_small.filter(col('BOROUGH') == 'QUEENS') \
                  .groupBy('CONTRIBUTING FACTOR VEHICLE 1') \
                  .agg(count('*').alias('count')) \
                  .orderBy(col('count').desc())

result.show()



+-----------------------------+-----+
|CONTRIBUTING FACTOR VEHICLE 1|count|
+-----------------------------+-----+
|                  Unspecified|26554|
|         Driver Inattentio...|15937|
|         Failure to Yield ...| 6322|
|             Backing Unsafely| 3673|
|         Following Too Clo...| 2708|
|         Passing or Lane U...| 1954|
|          Passing Too Closely| 1758|
|         Traffic Control D...| 1590|
|           Turning Improperly| 1544|
|              Fatigued/Drowsy| 1275|
|              Other Vehicular| 1199|
|          Driver Inexperience| 1068|
|          Alcohol Involvement|  975|
|                 Unsafe Speed|  954|
|         Unsafe Lane Changing|  678|
|            Pavement Slippery|  668|
|         Prescription Medi...|  582|
|           Lost Consciousness|  577|
|         View Obstructed/L...|  569|
|          Physical Disability|  551|
+-----------------------------+-----+
only showing top 20 rows



# Question 4

> _What is the average number or injuries for specific cars driving in specific suburbs_


In [ ]:
nyc_small.groupby(['VEHICLE TYPE CODE 1', 'BOROUGH'])['NUMBER OF PERSONS INJURED'].mean().sort_values(ascending=False).head(3)

In [ ]:
## Spark?

from pyspark.sql.functions import col, mean

result = sdf_small.groupBy('VEHICLE TYPE CODE 1', 'BOROUGH') \
                  .agg(mean('NUMBER OF PERSONS INJURED').alias('avg_injuries')) \
                  .orderBy(col('avg_injuries').desc()) \
                  .limit(3)

result.show()

+-------------------+--------+------------+
|VEHICLE TYPE CODE 1| BOROUGH|avg_injuries|
+-------------------+--------+------------+
|                rmb|  QUEENS|        11.0|
|                 PC|BROOKLYN|         5.0|
|          AMBULANVE|   BRONX|         4.0|
+-------------------+--------+------------+

